Expert System


In [ ]:
!pip install experta

In [ ]:
from experta import *
import numpy as np

In [ ]:
# we will create fact depending on the key_values
# way 1
f=Fact(a=1,b=2)
f['a']

1

In [ ]:
#create fact without keys(only values)
# This will create numeric index for your values
# way 2
f=Fact('x','y','z')
f[0]

'x'

In [ ]:
# we can merge numeric values with key_values
# but numeric values must be declared first
f=Fact('x','y','z',a=1,b=2)
# using index to access the fact
print(f[1],'',f['b'])

y  2


In [ ]:
#define Facts
class Alert (Fact):
    """ The alert level!."""
    pass
class Status (Fact):
    """ The system status!."""
    pass
f1=Alert('red')
f2=Status('critical')
print(f1,'\n',f2)

<Undeclared Fact> Alert('red') 
 <Undeclared Fact> Status('critical')


In [ ]:
#define Rules
class MyFact(Fact):
    pass
@Rule(MyFact())       #this is the LHS
def match_with_every_myfact():
    """ this rule will match with every instance of "MyFact"""
    # this is the RHS
    pass
@Rule(Fact('animal',family='felinae'))
def match_with_cats():
    """ match with every 'Fact' which:
    f[0]=='animal'
    f['family']=='felinea'
    """
    print('Meow!')

In [ ]:
# DefFacts for a set of facts
@DefFacts()
def needed_data():
    yield Fact(best_color='red')
    yield Fact(best_body='medium')
    yield Fact(best_sweetness='dry')

In [ ]:
# Knowledge Engine
engine=KnowledgeEngine()
engine.reset()
engine.declare(Fact(color='blue'))

Fact(color='blue')

In [ ]:
engine.facts

FactList([(0, InitialFact()), (1, Fact(color='blue'))])

In [ ]:
# modify some facts in the Knowledge Engine
engine.modify(engine.facts[1],_0='yellow',blink=True)

Fact(color='blue', 'yellow', blink=True)

In [ ]:
# show the facts after they have been modified.
engine.facts

FactList([(0, InitialFact()), (2, Fact(color='blue', 'yellow', blink=True))])

In [ ]:
# simple example about traffic light
from random import choice
from experta import *

class Light(Fact):
    """Information about the traffic light."""
    pass

class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print('Walk')
    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't Walk")
    @Rule(AS.light <<Light(color=L('yellow')|L('blinking-yellow')))
    def cautious(self,light):
        print("Be cautious because light is",light['color'])

engine=RobotCrossStreet()
engine.reset()
color_now=choice(['green','yellow','blinking-yellow','red'])
print('color_now:',color_now)
engine.declare(Light(color=color_now))
engine.run()
engine.facts

color_now: yellow
Be cautious because light is yellow


FactList([(0, InitialFact()), (1, Light(color='yellow'))])

In [ ]:
# simple example about animal
class Animal(Fact):
    """Info about animal type"""
    pass

class AnimalKE1(KnowledgeEngine):
    @Rule(Animal('cat'))
    def cat_sound(self):
        print('meow')
    @Rule(Animal('duck'))
    def duck_sound(self):
        print('quack')
        
engine=AnimalKE1()
engine.reset()
animal_type=choice(['cat','duck','horse'])
print('animal:',animal_type)
engine.declare(Animal(animal_type))
engine.run()
engine.facts

animal: cat
meow


FactList([(0, InitialFact()), (1, Animal('cat'))])

In [ ]:
class Animal(Fact):
    """Info about animal type"""
    pass

class AnimalKE2(KnowledgeEngine):
    @Rule(AND(Animal(has='feathers')),Animal(has='webbed-feet'))
    def guess_animal(self):
        print('Animal is duck')
        print('Sound is quack')
        
engine=AnimalKE2()
engine.reset()
engine.declare(Animal(has='feathers'),Animal(has='webbed-feet'))
engine.run()

Animal is duck
Sound is quack


In [ ]:
# conclude a fact
class Animal(Fact):
    """Info about animal type"""
    pass

class AnimalKE3(KnowledgeEngine):
    @Rule(AND(Animal(has='feathers'),Animal(has='webbed-feet')))
    def guess_animal(self):
        self.declare(Animal('duck'))
        print(self.facts)
    @Rule(Animal('duck'))
    def it_is_duck(self):
        print('Animal is duck')
        print("Sound is quack")
        
engine=AnimalKE3()
engine.reset()
engine.declare(Animal(has='feathers'),Animal(has='webbed-feet'))
engine.run()

<f-0>: InitialFact()
<f-1>: Animal(has='feathers')
<f-2>: Animal(has='webbed-feet')
<f-3>: Animal('duck')
Animal is duck
Sound is quack
